In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
training_file = pd.read_csv('/Users/bhavyaalekhya/Desktop/Courses/Fall_2022/CS6375/Homeworks/HW2/netflix/TrainingRatings.txt',names=['MovieID','UserID','Rating'],header=None)

In [3]:
movieids = np.unique(training_file['MovieID'].values)
userids = np.unique(training_file['UserID'].values)

In [4]:
training_set = pd.DataFrame(columns = movieids,index=userids)

In [5]:
for i in range(len(userids)):
    movies = training_file.query("UserID=={0}".format(userids[i])).MovieID.values
    ratings = training_file.query("UserID=={0}".format(userids[i])).Rating.values
    for j in range(len(movies)):
        training_set.iloc[i][movies[j]] = ratings[j]       

In [75]:
training_set = training_set.fillna(0) 

In [76]:
votes_of_users = {}
for i in range(len(userids)):
    if userids[i] not in votes_of_users.keys():
        total_movies = training_set.iloc[[i]].astype(bool).sum(axis=1).values[0]
        votes_of_users[userids[i]] = (np.sum(training_set.iloc[[i]].values))/(total_movies)

In [82]:
training_matrix2 = csr_matrix(training_set).toarray()

In [78]:
training_matrix = np.zeros(shape=(len(userids),len(movieids)))

In [83]:
for i in range(len(training_matrix)):
    
    training_matrix[i] = [x - votes_of_users[userids[i]] if x!=0.0 else x for x in training_matrix2[i]]

In [92]:
tm = np.cov(training_matrix2)

In [93]:
diag = np.diag(tm)

In [96]:
corrcoef = tm / np.sqrt(diag[:np.newaxis] * diag[np.newaxis : ])

In [23]:
testing_file = pd.read_csv("/Users/bhavyaalekhya/Desktop/Courses/Fall_2022/CS6375/Homeworks/HW2/netflix/TestingRatings.txt",names=['MovieId','UserID','Rating'],header=None)

In [24]:
y = testing_file["Rating"].values
X = testing_file[testing_file.columns[:2]].values

In [25]:
y_pred = np.zeros(shape=(len(y)))
movie_matrix = training_matrix.T

In [26]:
movie_matrix1 = pd.DataFrame(movie_matrix,columns=userids,index=movieids)

In [98]:
def sum_users(a1,j1,v,training_matrix):
    ttl = 0 
    den = 0
    for v1 in v:
        if v1!=a1:
            if v1>a1:
                ttl += corrcoef[a1][v1] * training_matrix[v1][j1]
                den += corrcoef[a1][v1]
            else:
                ttl += corrcoef[v1][a1] * training_matrix[v1][j1]
                den += corrcoef[v1][a1]

    rem = np.divide(ttl,den,where=den!=0.0)
    return rem

In [99]:
for i in range(len(X)):
    j = X[i][0]
    a = X[i][1]
    p_aj = 0
    v = np.nonzero(movie_matrix1.loc[j].values)[0]
    a1 = np.where(userids==a)[0][0]
    j1 = np.where(movieids==j)[0][0]
    p_aj = votes_of_users[a] + sum_users(a1,j1,v,training_matrix)
    
    y_pred[i] = p_aj

In [100]:
#Root Mean Squared Error
mse = mean_squared_error(y,y_pred)

rmse = np.sqrt(mse)

print("Root Mean Squared Error: %.3f"%rmse)

#Mean Absolute Error
mae = mean_absolute_error(y,y_pred)

print("Mean Absolute Error: %.3f"%mae)

Root Mean Squared Error: 0.900
Mean Absolute Error: 0.707
